In [1]:
pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 27.6 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Kavin\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
#to do things , tokenization , build vocabulary from the book "preprocessing

In [3]:
import re
import nltk
from collections import Counter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')  

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kavin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Kavin\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kavin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Kavin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Kavin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kavin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-d

True

In [9]:
def build_vocab_model(text):
    # 1. Cleaning with 're'
    text = text.lower()
    words = re.findall(r'\b[a-z]{2,}\b', text) # Remove punctuation and single letters
    
    lemmatizer = WordNetLemmatizer()
    tagged_words = nltk.pos_tag(words)
    
    # Frequency Distribution
    freq_dist = Counter(words)
    
    vocabulary = {
        "basic_everyday": set(),
        "high_freq_polysemous": set(),
        "academic_subject_specific": set()
    }

    for word, tag in tagged_words:
        # Get synsets to check for multiple meanings (Polysemy)
        synsets = wordnet.synsets(word)
        meaning_count = len(synsets)
        count = freq_dist[word]

        # Logic for High-frequency words with multiple meanings
        # (Appears > 10 times and has 5+ definitions in WordNet)
        if count > 10 and meaning_count > 5:
            vocabulary["high_freq_polysemous"].add(word)
        
        # Logic for Subject-specific/Academic terms
        # (Nouns/Verbs that are rarer in this specific text but have specific definitions)
        elif tag in ['NN', 'NNS'] and len(word) > 7 and count < 5:
            vocabulary["academic_subject_specific"].add(word)
            
        # Basic everyday words
        # (Common nouns/verbs with moderate frequency)
        elif count > 5:
            vocabulary["basic_everyday"].add(word)

    return vocabulary

with open('alice_in_wonderland_eng.txt', 'r') as f:
     data = f.read()
     model = build_vocab_model(data)
     print(f"Basic Words (sample): {list(model['basic_everyday'])[:5]}")
     print(f"Polysemous Words (sample): {list(model['high_freq_polysemous'])[:5]}")
     print(f"Academic/Specific (sample): {list(model['academic_subject_specific'])[:5]}")

Basic Words (sample): ['walk', 'when', 'from', 'answer', 'may']
Polysemous Words (sample): ['good', 'is', 'just', 'as', 'take']
Academic/Specific (sample): ['silhouettes', 'directions', 'muchness', 'messenger', 'flamingoes']


In [8]:
def print_high_frequency_words(text, top_n=20):
    # 1. Clean and tokenize
    text = text.lower()
    words = re.findall(r'\b[a-z]{2,}\b', text)
    
    # 2. Calculate frequency
    freq_dist = Counter(words)
    
    # 3. Get the most common words
    most_common = freq_dist.most_common(top_n)
    
    print(f"--- Top {top_n} High-Frequency Words ---")
    print(f"{'Word':<15} | {'Frequency':<10}")
    print("-" * 30)
    for word, count in most_common:
        print(f"{word:<15} | {count:<10}")

# Usage
with open('alice_in_wonderland_eng.txt', 'r', encoding='utf-8') as f:
    data = f.read()
    print_high_frequency_words(data)

--- Top 20 High-Frequency Words ---
Word            | Frequency 
------------------------------
the             | 393       
you             | 295       
and             | 253       
alice           | 239       
to              | 211       
it              | 210       
of              | 142       
in              | 98        
that            | 89        
what            | 77        
queen           | 71        
is              | 61        
at              | 59        
with            | 58        
all             | 57        
if              | 57        
know            | 57        
they            | 54        
turtle          | 54        
oh              | 53        
